In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from torch import nn
from torch import optim
from sklearn.preprocessing import LabelEncoder
filename = "/kaggle/input/tabular-playground-series-jun-2021/train.csv"
test_filename="/kaggle/input/tabular-playground-series-jun-2021/test.csv"
traindf = pd.read_csv(filename,index_col='id')
testdf=pd.read_csv(test_filename,index_col='id')
lencoder = LabelEncoder()
y = lencoder.fit_transform(traindf['target']).astype('int64')
testdf.tail()



In [ ]:
print(y)
print(type(y))
traindf['target']=y
traindf.head()

In [ ]:
inputSize = len(traindf.columns) -1
numClasses = traindf['target'].nunique()
batchSize = 1024

In [ ]:
len(traindf.columns) -1
print(numClasses)
columns = list(traindf.columns)
columns.pop()
columns

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
traindf[columns] =scaler.fit_transform(traindf[columns])
traindf.head()


In [ ]:
trainLoader = torch.utils.data.DataLoader(dataset=torch.tensor(traindf.values), batch_size=batchSize, shuffle=True)

In [ ]:
learningRate=0.03

In [ ]:
model = nn.Sequential(nn.Linear(inputSize,64),
                      nn.ReLU(),
                      nn.Linear(64,32),
                      nn.ReLU(),
                      nn.Linear(32,16),
                      nn.ReLU(),
                      nn.Linear(16,numClasses),
                      nn.LogSoftmax(dim=1))
              
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)  

In [ ]:
epoch=20
for e in range(epoch):
  running_loss=0
  for i,data in enumerate(trainLoader):
    labels=data[:,-1]
    attributes=data[:,0:len(traindf.columns) -1].float()
    logits=model(attributes)
    loss= criterion(logits,labels.long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  else:
    print(f"Training loss: {running_loss/len(trainLoader)}")

In [ ]:
testdf.head()

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i,data in enumerate(trainLoader):
        labels=data[:,-1]
        attributes=data[:,0:len(traindf.columns) -1].float()
        outputs = model(attributes)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')


In [ ]:
sample=pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
sample.head()

In [ ]:
print(sample.head())
print(sample.shape)
print(testdf.shape)
print(traindf.shape)

In [ ]:
scaler2=StandardScaler()
testdf[columns] =scaler.fit_transform(testdf[columns])
testdf.head()

In [ ]:
testLoader = torch.utils.data.DataLoader(dataset=torch.tensor(testdf.values), batch_size=batchSize, shuffle=False)

In [ ]:
with torch.no_grad():
    a=torch.empty(0,9)
    for i,data in enumerate(testLoader):
        attributes=data.float()
        logps = model(attributes)
        ps = torch.exp(logps)
        ps.shape
        a=torch.vstack((a,ps))

In [ ]:
print(a.shape)

In [ ]:
result= a.numpy()
sample.iloc[:,1:10]=result
sample.head()


In [ ]:
sample.to_csv('/kaggle/working/output.csv',index=False)